In [3]:
# 📦 Install dependencies (if not already)
!pip install git+https://github.com/facebookresearch/segment-anything.git
!pip install gradio nibabel opencv-python numpy

# 📚 Imports
import gradio as gr
import nibabel as nib
import numpy as np
import cv2
from segment_anything import sam_model_registry, SamPredictor

# 🔧 Load SAM Model
sam = sam_model_registry["vit_b"](checkpoint="/kaggle/input/sam_vit_b_01ec64.pth/pytorch/sam_vit_b_01ec64.pth/1/sam_vit_b_01ec64.pth")
predictor = SamPredictor(sam)

# 🧠 Global Storage for Uploaded Image
uploaded_image = {}

# STEP 1: Preview middle slice from .nii file
def preview_nifti(file_obj):
    nii = nib.load(file_obj.name)
    data = nii.get_fdata()
    slice_2d = data[:, :, data.shape[2] // 2]
    norm = cv2.normalize(slice_2d, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
    image_rgb = cv2.cvtColor(norm, cv2.COLOR_GRAY2RGB)
    uploaded_image['image'] = image_rgb  # store for next step
    return image_rgb

# STEP 2: Segment using bounding box
def segment_uploaded_image(x1, y1, x2, y2):
    image_rgb = uploaded_image.get('image')
    if image_rgb is None:
        return None, 0.0, "No image uploaded."

    predictor.set_image(image_rgb)
    input_box = np.array([[x1, y1, x2, y2]])
    masks, scores, logits = predictor.predict(box=input_box, multimask_output=False)
    mask = masks[0]
    confidence_score = float(scores[0])

    result = image_rgb.copy()
    result[mask] = [255, 0, 0]  # 🟥 RED MASK
    text = f"Model Confidence (IOU Score): {confidence_score:.4f}"
    return result, confidence_score, text

# 🎨 Gradio Interface
with gr.Blocks() as app:
    gr.Markdown("## 🧠 Brain Tumor Segmentation using SAM")
    gr.Markdown("Upload a `.nii` MRI file. Preview the middle slice. Set a bounding box to segment tumor region.")

    file_input = gr.File(label="Upload .nii MRI Image")

    with gr.Row():
        preview_btn = gr.Button("Preview Image")
    preview_output = gr.Image(label="Middle Slice Preview")

    with gr.Row():
        x1 = gr.Slider(0, 255, value=60, label="x1")
        y1 = gr.Slider(0, 255, value=60, label="y1")
        x2 = gr.Slider(0, 255, value=180, label="x2")
        y2 = gr.Slider(0, 255, value=180, label="y2")

    segment_btn = gr.Button("Submit")
    segment_output = gr.Image(label="Segmented Output")

    with gr.Column():
        confidence_bar = gr.Slider(0.0, 1.0, value=0.0, label="Confidence Score (IOU)", interactive=False)
        confidence_text = gr.Textbox(label="Confidence Interpretation", interactive=False)

    preview_btn.click(fn=preview_nifti, inputs=file_input, outputs=preview_output)
    segment_btn.click(fn=segment_uploaded_image, inputs=[x1, y1, x2, y2], outputs=[segment_output, confidence_bar, confidence_text])

app.launch()


  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-5yiv6x35
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-5yiv6x35
  Resolved https://github.com/facebookresearch/segment-anything.git to commit dca509fe793f601edb92606367a655c15ac00fdf
  Preparing metadata (setup.py) ... done
* Running on local URL:  http://127.0.0.1:7860
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://7ff0017b846b232cfd.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
